Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.
- BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [ ]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


In [ ]:
# perform tuning and cross validation here

In [4]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

In [27]:
data = pd.read_csv('model_df.csv')

In [30]:
from sklearn.model_selection import KFold

#Split the data into 5 folds
def custom_cross_validation(training_data, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    training_folds = []
    validation_folds = []

    for train_index, val_index in kf.split(training_data):
        train_fold = training_data.iloc[train_index]
        val_fold = training_data.iloc[val_index]

        # Append folds to the respective lists
        training_folds.append(train_fold)
        validation_folds.append(val_fold)

    return training_folds, validation_folds

In [31]:
from sklearn.metrics import mean_absolute_error
from itertools import product

def hyperparameter_search(training_folds, validation_folds, param_grid, evaluation_metric):
    best_hyperparameters = None
    best_score = float('-inf')  # Initialize with a very low value

    # Generate all combinations of hyperparameters
    hyperparameter_combinations = list(product(*param_grid.values()))

    for hyperparams in hyperparameter_combinations:
        fold_scores = []
        
        # Create and train the model using the current hyperparameters
        for i in range(len(training_folds)):
            model = RandomForestRegressor(**dict(zip(param_grid.keys(), hyperparams)))
            model.fit(training_folds[i].drop(columns=['description.sold_price']), training_folds[i]['description.sold_price'])

            # Make predictions on the validation set
            predictions = model.predict(validation_folds[i].drop(columns=['description.sold_price']))
            score = evaluation_metric(validation_folds[i]['description.sold_price'], predictions)

            fold_scores.append(score)

        # Calculate the average score across all folds
        average_score = sum(fold_scores) / len(fold_scores)
        
        # Update best hyperparameters if the current combination is better
        if average_score > best_score:
            best_score = average_score
            best_hyperparameters = hyperparams

    return best_hyperparameters

In [33]:
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

output = custom_cross_validation(data, n_splits=5)
best_hyperparameters = hyperparameter_search(output[0], output[1], param_grid=param_grid, evaluation_metric=mean_absolute_error)

print("Best Hyperparameters:", best_hyperparameters)

Best Hyperparameters: (10, 10, 4, 'log2')


We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [ ]:
# save your best model here

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [35]:
#Assuming the order of hyperparameters in the tuple is known and fixed
hyperparameter_names = ['max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features']
best_hyperparameters_dict = dict(zip(hyperparameter_names, best_hyperparameters))

print(best_hyperparameters_dict)

#Load the data
data = pd.read_csv('model_df.csv')

#Set 'y' as the target variable
y = data['description.sold_price']

#Set 'X' as all other columns except the target variable
X = data.drop(columns=['description.sold_price'])

{'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2'}


In [38]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import pickle
import os

#Use the newly created dictionary for hyperparameters
pipeline_steps = [

    ('regressor', RandomForestRegressor(max_depth=best_hyperparameters_dict['max_depth'],
                                        min_samples_split=best_hyperparameters_dict['min_samples_split'],
                                        min_samples_leaf=best_hyperparameters_dict['min_samples_leaf'],
                                        max_features=best_hyperparameters_dict['max_features']))
]

#Create and fit the pipeline
pipeline = Pipeline(steps=pipeline_steps)
pipeline.fit(X, y)  # Assuming 'X' and 'y' are your features and target variable

#Save the pipeline
if not os.path.exists('models'):
    os.makedirs('models')

with open('models/final_model_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here